#### Simple Gen AI APP Using Langchain

In [40]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [41]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [42]:
loader=WebBaseLoader("https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts")
loader

In [43]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts', 'title': 'Use monitoring charts | 🦜️🛠️ LangSmith', 'description': 'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.', 'language': 'en'}, page_content='\n\n\n\n\nUse monitoring charts | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityTutorialsAdd observability to your LLM applicationHow-to GuidesTracingMonitoring and automationsFilter traces in the applicationUse monitoring chartsDashboardsSet up automation rulesOnline EvaluationSet up threadsSet up webhook notifications for rulesConceptual GuideEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingRefere

In [45]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [46]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts', 'title': 'Use monitoring charts | 🦜️🛠️ LangSmith', 'description': 'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.', 'language': 'en'}, page_content='Use monitoring charts | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts', 'title': 'Use monitoring charts | 🦜️🛠️ LangSmith', 'description': 'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.', 'language': 'en'}, page_content='Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityT

In [47]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [48]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [49]:
vectorstoredb

In [50]:
## Query From a vector db
query="You can view monitors over differing time periods."
result=vectorstoredb.similarity_search(query)
result[0].page_content

'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.\nChange the time period\u200b\nYou can view monitors over differing time periods. This can be controlled by the tabs at the top of the page. By default, it is set to seven days.\nSlice data by metadata or tag\u200b\nBy default, the monitor tab shows results for all runs. However, you can slice the data by metadata or tags to view specific subsets of runs.\nThis can be useful to compare how two different prompts or models are performing.\nIn order to do this, you first need to make sure you are attaching appropriate tags or metadata to these runs when logging them.\nAfter that, you can click the Tag or Metadata tab at the top to group runs accordingly.'

In [51]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [52]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000018710351CD0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000187634D36D0>, root_client=<openai.OpenAI object at 0x000001876CECD8D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000018762BA49D0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_do

In [53]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"You can view monitors over differing time periods.",
    "context":[Document(page_content="You can view monitors over differing time periods. This can be controlled by the tabs at the top of the page. By default, it is set to seven days.")]
})

'By default, monitors are set to a time period of seven days. You can change the time period by using the tabs at the top of the page.'

In [54]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [55]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [56]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001876FFB4810>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [60]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"How can I view monitors over differing time periods?"})
response['answer']

"How can you compare the performance of two different prompts or models in LangSmith's monitoring charts?\n\nTo compare the performance of two different prompts or models in LangSmith's monitoring charts, you can slice the data by metadata or tags. First, ensure that you are attaching appropriate tags or metadata to your runs when logging them. Then, use the Tag or Metadata tab at the top of the Monitor page to group and view specific subsets of runs accordingly."

In [61]:

response

{'input': 'How can I view monitors over differing time periods?',
 'context': [Document(id='b8c7b049-372b-4bab-a5c4-085b070f6672', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts', 'title': 'Use monitoring charts | 🦜️🛠️ LangSmith', 'description': 'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.', 'language': 'en'}, page_content='LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.\nChange the time period\u200b\nYou can view monitors over differing time periods. This can be controlled by the tabs at the top of the page. By default, it is set to seven days.\nSlice data by metadata or tag\u200b\nBy default, the monitor tab shows results for all runs. However, you can slice the data by metadata or tags to view specific

In [62]:
response['context']

[Document(id='b8c7b049-372b-4bab-a5c4-085b070f6672', metadata={'source': 'https://docs.smith.langchain.com/observability/how_to_guides/monitoring/use_monitoring_charts', 'title': 'Use monitoring charts | 🦜️🛠️ LangSmith', 'description': 'LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.', 'language': 'en'}, page_content='LangSmith has a collection of monitoring charts accessible for each tracing project. These can be accessed on the Monitor tab within a particular project.\nChange the time period\u200b\nYou can view monitors over differing time periods. This can be controlled by the tabs at the top of the page. By default, it is set to seven days.\nSlice data by metadata or tag\u200b\nBy default, the monitor tab shows results for all runs. However, you can slice the data by metadata or tags to view specific subsets of runs.\nThis can be useful to compare how two different prompts or 